In [1]:
# ! pip install plotly==5.22.0
# ! pip install jupyterlab-widgets==1.1.1 ipywidgets==7.7.2
# #! pip install jupyterlab-widgets==3.0.11 ipywidgets==8.1.3
# ! pip install umap-learn
! jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

a = ana.Anatomist()

In [4]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
import umap
import os

In [5]:
#! jupyter labextension list
#! jupyter nbextension list

### In this example, the embeddings are generated from a Barlow Twin model trained on the Left CINGULATE region on UKB for the UKB and HCP datasets.

 -For the HCP dataset, the associated buckets can be found in: 

/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Lbuckets

 -For the UKB dataset, the buckets are not generated yet, but it would be found in:

/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE/mask/Lbuckets

### Load your data

In [7]:
#embeddings_HCP = pd.read_csv("/neurospin/dico/adufournet/Runs/02_Heritability_Left_PCS_HCP/Output/2024-05-13/09-33-29_206/hcp_epoch60_embeddings/full_embeddings.csv", index_col=0)
#embeddings_UKB = pd.read_csv("/neurospin/dico/adufournet/Runs/02_Heritability_Left_PCS_HCP/Output/2024-05-13/09-33-29_206/UKB_epoch60_embeddings/full_embeddings.csv", index_col=0)

#embeddings_HCP = pd.read_csv("/neurospin/dico/adufournet/Runs/08_Heritability_Left_Orbital_HCP_dim256/Output/20-56-02_1/HCP_random_epoch70_embeddings/full_embeddings.csv", index_col=0)
#embeddings_UKB = pd.read_csv("/neurospin/dico/adufournet/Runs/08_Heritability_Left_Orbital_HCP_dim256/Output/20-56-02_1/UKB_random_epoch70_embeddings/full_embeddings.csv", index_col=0)

embeddings_HCP = pd.read_csv("/neurospin/dico/adufournet/Runs/04_Heritability_Right_PCS_HCP_dim10/Output/2024-05-29/19-16-32_3/HCP_random_epoch40_embeddings/full_embeddings.csv", index_col=0)
embeddings_UKB = pd.read_csv("/neurospin/dico/adufournet/Runs/04_Heritability_Right_PCS_HCP_dim10/Output/2024-05-29/19-16-32_3/UKB_random_epoch40_embeddings/full_embeddings.csv", index_col=0)
embeddings_UKB.head()

embeddings_HCP = embeddings_HCP
embeddings_UKB = embeddings_UKB
embeddings_UKB.head()

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10
ID,,,,,,,,,,
sub-1000021,7.625430,-110.843796,-31.410776,-14.304582,-58.193040,1.267348,-16.256983,-31.085651,-6.734729,-37.349705
sub-1000458,-2.686900,-99.696810,-16.900297,-11.260241,-49.261047,-40.481380,4.340034,-11.678871,-14.759934,-19.288872
sub-1000575,-3.391189,-41.532593,-16.283690,6.923420,-15.241653,-6.485703,4.359082,-1.681517,-10.869176,0.104506
sub-1000606,1.382976,-89.879420,-1.775408,18.353903,-51.242970,-12.489634,-2.822808,2.998624,22.957664,-6.521338
sub-1000963,11.553831,-154.482510,-12.179215,-20.130846,-81.294730,73.374030,3.920644,-44.567265,29.138690,-33.605183


### Scale your data

In [8]:
scaler = StandardScaler()
scaler.fit(embeddings_UKB)

scl_bdd_hcp = scaler.transform(embeddings_HCP)
scl_bdd_ukb = scaler.transform(embeddings_UKB)

### 2D UMAP

In [9]:
reducer2D = umap.UMAP(n_components=2)

reducer2D.fit(scl_bdd_ukb)

bdd_2D_HCP = reducer2D.transform(scl_bdd_hcp)
bdd_2D_UKB = reducer2D.transform(scl_bdd_ukb)

/casa/home/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


In [10]:
bdd_2D_HCP = pd.DataFrame(bdd_2D_HCP, columns=['Dim 1', 'Dim 2'])
bdd_2D_UKB = pd.DataFrame(bdd_2D_UKB, columns=['Dim 1', 'Dim 2'])

bdd_2D_HCP['Dataset'] = 'hcp'
bdd_2D_UKB['Dataset'] = 'UkBioBank'

bdd_2D_HCP['ID'] = embeddings_HCP.index
bdd_2D_UKB['ID'] = embeddings_UKB.index

bdd_2D_All = pd.concat([bdd_2D_UKB,bdd_2D_HCP], axis=0)
bdd_2D_All.head()

,Dim 1,Dim 2,Dataset,ID
0,6.702684,4.399197,UkBioBank,sub-1000021
1,-0.267239,1.195124,UkBioBank,sub-1000458
2,-4.141913,4.709678,UkBioBank,sub-1000575
3,0.349915,6.150640,UkBioBank,sub-1000606
4,9.541675,4.892012,UkBioBank,sub-1000963


### Plot it in the notebook or write a html file

In [19]:
subject_id_list = []
dataset_name_list = []

In [20]:
# Create the scatter plot using plotly express
fig = px.scatter(
    bdd_2D_All, x='Dim 1', y='Dim 2', 
    color='Dataset',
    title='2D UMAP HCP and UKB',
    labels={'0': 'dim 1', '1': 'dim 2'},
    hover_data= ['Dataset', 'ID'],
    opacity=0.5,
    width=800, height=600
)

#fig.show()

# Convert the figure to a FigureWidget
f = go.FigureWidget(fig)

# Define the callback function
def click_callback(trace, points, selector):
    for trace_index in range(len(f.data)):
        if trace_index == points.trace_index:
            customdata = f.data[trace_index].customdata
            for i in points.point_inds:
                point_dataset, point_id = customdata[i]
                print(f"Clicked point ID: {point_id}, Dataset: {point_dataset}")
                subject_id_list.append(point_id)
                dataset_name_list.append(point_dataset)

# Attach the callback to the on_click event for all traces
for trace in f.data:
    trace.on_click(click_callback)

# Display the figure widget
f

FigureWidget({
    'data': [{'customdata': array([['UkBioBank', 'sub-1000021'],
                                   ['UkBioBank', 'sub-1000458'],
                                   ['UkBioBank', 'sub-1000575'],
                                   ...,
                                   ['UkBioBank', 'sub-6023847'],
                                   ['UkBioBank', 'sub-6024038'],
                                   ['UkBioBank', 'sub-6024754']], dtype=object),
              'hovertemplate': ('Dataset=%{customdata[0]}<br>Di' ... '{customdata[1]}<extra></extra>'),
              'legendgroup': 'UkBioBank',
              'marker': {'color': '#636efa', 'opacity': 0.5, 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'UkBioBank',
              'showlegend': True,
              'type': 'scattergl',
              'uid': '1a3e14e5-b922-42c6-824a-e592ff01b34c',
              'x': array([ 6.702684  , -0.26723868, -4.141913  , ...,  3.5206687 ,  1.4711552 ,
                 

Clicked point ID: sub-4662034, Dataset: UkBioBank
Clicked point ID: sub-4168749, Dataset: UkBioBank
Clicked point ID: sub-2815300, Dataset: UkBioBank
Clicked point ID: sub-2302098, Dataset: UkBioBank


In [21]:
print(subject_id_list, dataset_name_list)

['sub-4662034', 'sub-4168749', 'sub-2815300', 'sub-2302098'] ['UkBioBank', 'UkBioBank', 'UkBioBank', 'UkBioBank']


In [11]:
#subject_id_list = [214423, 849971, 208630, 683256, 589567]
#dataset_name_list = ['hcp' for i in range(5)]

In [22]:
side = "L"
region = "CINGULATE" #"ORBITAL" #"CINGULATE"

bucket_files = []

for subject_id, dataset in zip(subject_id_list,dataset_name_list):
    if dataset.lower() in ['ukb', 'ukbiobank']:
        dataset = 'UkBioBank'
        path = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops'

    if dataset.lower() in ['hcp']:
        path = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset.lower()}/crops/2mm/{region}/mask/{side}crops'

    filename = f'{path}/{subject_id}_cropped_skeleton.nii.gz'#.minf'

    if os.path. isfile(filename):
        bucket_files.append(filename)
    else:
        print(f"{filename} is not a correct path, or the .bck doesn't exist")
bucket_files

['/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE/mask/Lcrops/sub-4662034_cropped_skeleton.nii.gz',
 '/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE/mask/Lcrops/sub-4168749_cropped_skeleton.nii.gz',
 '/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE/mask/Lcrops/sub-2815300_cropped_skeleton.nii.gz',
 '/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE/mask/Lcrops/sub-2302098_cropped_skeleton.nii.gz']

In [23]:
from soma import aims

def to_bucket(obj):
    if obj.type() == obj.BUCKET:
        return obj
    avol = a.toAimsObject(obj)
    c = aims.Converter(intype=avol, outtype=aims.BucketMap_VOID)
    abck = c(avol)
    bck = a.toAObject(abck)
    bck.releaseAppRef()
    return bck

def build_gradient(pal):
    gw = ana.cpp.GradientWidget(None, 'gradientwidget', pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0, :3][:, ::-1]  # BGRA -> RGBA
    pal.update()

In [24]:
block = a.createWindowsBlock(3) # nb of columns
d = {}

for i, file in enumerate(bucket_files):
    d[f'bck_{i}'] = to_bucket(a.loadObject(file))
    d[f'w_{i}'] = a.createWindow('3D', block=block)#geometry=[100+400*(i%3), 100+440*(i//3), 400, 400])
    d[f'w_{i}'].addObjects(d[f'bck_{i}'])

In [25]:
dic_vol = {}
dim = 0
rep = 0
while dim == 0 and rep < len(subject_id_list):
    mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset_name_list[rep]}/crops/2mm/{region}/mask/{side}crops"
    if os.path. isfile(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz'):
        sum_vol = aims.read(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz').astype(float)
        dim = sum_vol.shape
        sum_vol.fill(0)
    else: 
        print(f'FileNotFound {mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz')
        #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id_list[0]}_cropped_skeleton.nii.gz')
    rep += 1

for subject_id, dataset in zip(subject_id_list,dataset_name_list):
    if dataset.lower() in ['ukb', 'ukbiobank']:
        dataset = 'UkBioBank'
    elif dataset.lower() == 'hcp':
        dataset = 'hcp'
        
    mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"

    if os.path. isfile(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz'):
        vol = aims.read(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
        # compare the dim with the first file dim

        if vol.np.shape != dim:
            raise ValueError(f"{subject_id_list[0]} and {subject_id} must have the same dim")

            
        # to have a binary 3D structure
        dic_vol[subject_id] = (vol.np > 0).astype(int)
        sum_vol.np[:] += (vol.np > 0).astype(int) 
    else: 
        print(f'FileNotFound {mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
        #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')

sum_vol = sum_vol / len(subject_id_list)
print(sum_vol, sum_vol.shape)


a_sum_vol = a.toAObject(sum_vol)
a_sum_vol.setPalette(minVal=0, absoluteMode=True)
wsum = a.createWindow('Sagittal', block=block)
wsum.addObjects(a_sum_vol)
rvol = a.fusionObjects(objects=[a_sum_vol], method='VolumeRenderingFusionMethod')
rvol.releaseAppRef()
# custom palette
n = len(subject_id_list)
pal = a.createPalette('VR-palette')
pal.header()['palette_gradients'] = f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.5/n};0;1;1'
build_gradient(pal)
rvol.setPalette('VR-palette', minVal=0, absoluteMode=True)
pal2 = a.createPalette('slice-palette')
pal2.header()['palette_gradients'] = f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.3/n};0;{0.7/n};1;1;1'
build_gradient(pal2)
a_sum_vol.setPalette('slice-palette')
# rvol.palette().fill()
wvr = a.createWindow('3D', block=block)
wvr.addObjects(rvol)
#print(dic_vol[subject_id_list[0]].shape)
#print(np.count_nonzero(dic_vol[subject_id_list[0]]))

<soma.aims.Volume_DOUBLE object at 0x755b9c6f0dc0> (18, 41, 38, 1)


In [26]:
# Create axis
axes = list(dim[:3])
dim1 = list(dim[:3])[0]
dim2 = list(dim[:3])[1]
dim3 = list(dim[:3])[2]

# Create Data
data = sum_vol.np.reshape(list(dim[:3]))

X, Y, Z = np.mgrid[0:dim1:1, 0:dim2:1, 0:dim3:1]
values = np.flip(data, axis=[1,2])

fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=values.flatten(),
    isomin=0.1,
    isomax=1,
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=17, # needs to be a large number for good volume rendering
    ))
fig.show()

Position : 16.9379, 16.2868, 34.986, 0
Position : 16, 16.724, 52.375, 0
Position : 16, 17.151, 52.375, 0
Position : 16, 18.6458, 52.1615, 0
Position : 16, 32.3125, 39.5625, 0
Position : 16, 63.4896, 33.7969, 0
Position : 16, 17.7917, 52.5885, 0


In [ ]:
"""
data_filenames_str = ' '.join(data_filenames_list)

#os.system(f'anatomist --input {data_filenames_str}')

# Command to run the Python script inside the container
command = f'bv bash -c "python visu_anatomist.py {data_filenames_str}"'

# Execute the command
os.system(command)
"""

In [ ]:
# See for more information
# https://plotly.com/python/line-and-scatter/
# https://plotly.com/python/setting-graph-size/
# app.run_server(debug=True)